<a href="https://colab.research.google.com/github/Faithkoi-ai/HPD/blob/main/HDP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit

In [ ]:
import streamlit as st
# Title of the app
st.title('Preeclampsia prediction')

# Display text
st.write('Hypertension disorder in pregnant women app.')

# Add a slider widget
number = st.slider('Pick a number', 0, 100, 50)
st.write('You selected:', number)

In [ ]:
import warnings
warnings.simplefilter(action ="ignore")

from collections import Counter
import warnings
warnings.filterwarnings("ignore")

# Import the necessary packages
import numpy as np
import pandas as pd

In [ ]:
#load data
df=pd.read_csv('/content/Maternal Health Risk Data Set.csv')
df.head(10)

,Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate,RiskLevel
0,25,130,80,15.00,98.0,86,high risk
1,35,140,90,13.00,98.0,70,high risk
2,29,90,70,8.00,100.0,80,high risk
3,30,140,85,7.00,98.0,70,high risk
4,35,120,60,6.10,98.0,76,low risk
5,23,140,80,7.01,98.0,70,high risk
6,23,130,70,7.01,98.0,78,mid risk
7,35,85,60,11.00,102.0,86,high risk
8,32,120,90,6.90,98.0,70,mid risk
9,42,130,80,18.00,98.0,70,high risk


In [ ]:
# DataFrame information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1014 entries, 0 to 1013
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Age          1014 non-null   int64  
 1   SystolicBP   1014 non-null   int64  
 2   DiastolicBP  1014 non-null   int64  
 3   BS           1014 non-null   float64
 4   BodyTemp     1014 non-null   float64
 5   HeartRate    1014 non-null   int64  
 6   RiskLevel    1014 non-null   object 
dtypes: float64(2), int64(4), object(1)
memory usage: 55.6+ KB


In [ ]:
df.columns

Index(['Age', 'SystolicBP', 'DiastolicBP', 'BS', 'BodyTemp', 'HeartRate',
       'RiskLevel'],
      dtype='object')

In [ ]:
df.shape

(1014, 7)

In [ ]:
# Convert body temperature from Fahrenheit to Celsius
df["BodyTemp"] = (df["BodyTemp"] - 32) * 5/9
df.head()


,Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate,RiskLevel
0,25,130,80,15.0,36.666667,86,high risk
1,35,140,90,13.0,36.666667,70,high risk
2,29,90,70,8.0,37.777778,80,high risk
3,30,140,85,7.0,36.666667,70,high risk
4,35,120,60,6.1,36.666667,76,low risk


In [ ]:
# Analyse statically insight of data
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Age,1014.0,29.871795,13.474386,10.000000,19.000000,26.000000,39.000000,70.000000
SystolicBP,1014.0,113.198225,18.403913,70.000000,100.000000,120.000000,120.000000,160.000000
DiastolicBP,1014.0,76.460552,13.885796,49.000000,65.000000,80.000000,90.000000,100.000000
BS,1014.0,8.725986,3.293532,6.000000,6.900000,7.500000,8.000000,19.000000
BodyTemp,1014.0,37.036160,0.761880,36.666667,36.666667,36.666667,36.666667,39.444444
HeartRate,1014.0,74.301775,8.088702,7.000000,70.000000,76.000000,80.000000,90.000000


In [ ]:
      # Map RiskLevel to integer values
risk_mapping = {"low risk": 0, "mid risk": 1, "high risk": 2}
df["RiskLevel"] = df["RiskLevel"].map(risk_mapping)
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1014 entries, 0 to 1013
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Age          1014 non-null   int64  
 1   SystolicBP   1014 non-null   int64  
 2   DiastolicBP  1014 non-null   int64  
 3   BS           1014 non-null   float64
 4   BodyTemp     1014 non-null   float64
 5   HeartRate    1014 non-null   int64  
 6   RiskLevel    1014 non-null   int64  
dtypes: float64(2), int64(5)
memory usage: 55.6 KB


,Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate,RiskLevel
0,25,130,80,15.0,36.666667,86,2
1,35,140,90,13.0,36.666667,70,2
2,29,90,70,8.0,37.777778,80,2
3,30,140,85,7.0,36.666667,70,2
4,35,120,60,6.1,36.666667,76,0


In [ ]:
# Check for missing values in a specific column
print("\nMissing values in column:['Age', 'SystolicBP', 'DiastolicBP', 'BS', 'BodyTemp', 'HeartRate', 'RiskLevel'] :")
print(df[['Age', 'SystolicBP', 'DiastolicBP', 'BS', 'BodyTemp', 'HeartRate', 'RiskLevel']].isnull().sum())



Missing values in column:['Age', 'SystolicBP', 'DiastolicBP', 'BS', 'BodyTemp', 'HeartRate', 'RiskLevel'] :
Age            0
SystolicBP     0
DiastolicBP    0
BS             0
BodyTemp       0
HeartRate      0
RiskLevel      0
dtype: int64


In [ ]:
# Identify the outlier in HeartRate
df.HeartRate.sort_values().head()

,HeartRate
908,7
499,7
702,60
783,60
784,60


In [ ]:
# Remove the outlier in HeartRate
df = df.drop(df.index[df.HeartRate == 7])
df.head()

,Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate,RiskLevel
0,25,130,80,15.0,36.666667,86,2
1,35,140,90,13.0,36.666667,70,2
2,29,90,70,8.0,37.777778,80,2
3,30,140,85,7.0,36.666667,70,2
4,35,120,60,6.1,36.666667,76,0


In [ ]:
df.shape


(1012, 7)

In [ ]:
# Feature scaling
!pip install scikit-learn
import pandas as pd
from sklearn.preprocessing import StandardScaler  # Import StandardScaler
columns = ["Age", "DiastolicBP", "BS", "BodyTemp", "HeartRate", "SystolicBP"]
scale_X = StandardScaler()
X = pd.DataFrame(scale_X.fit_transform(df.drop(["RiskLevel"],axis = 1),), columns = columns)
y = df["RiskLevel"]

In [ ]:
print(y.isnull().sum())

0


In [ ]:
# train_test_split
from sklearn.model_selection import train_test_split

# Drop rows with NaN values in 'y'
y = y.dropna()

# Ensure X and y have the same number of samples
X = X.iloc[:len(y)]

# Now perform the split with the cleaned data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify = y)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((809, 6), (203, 6), (809,), (203,))

In [ ]:
X.head()

,Age,DiastolicBP,BS,BodyTemp,HeartRate,SystolicBP
0,-0.363796,0.913349,0.254566,1.903633,-0.485808,1.538308
1,0.378765,1.456516,0.974379,1.296644,-0.485808,-0.589878
2,-0.066772,-1.259323,-0.465247,-0.220828,0.972192,0.740239
3,0.007484,1.456516,0.614473,-0.524323,-0.485808,-0.589878
4,0.378765,0.370181,-1.185059,-0.797468,-0.485808,0.208192


In [ ]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import joblib

In [ ]:
# Ensure numerical types
df['SystolicBP'] = df['SystolicBP'].astype(float)
df['DiastolicBP'] = df['DiastolicBP'].astype(float)
df['Age'] = df['Age'].astype(float)
df['BS'] = df['BS'].astype(float)
df['HeartRate'] = df['HeartRate'].astype(float)
df['BodyTemp'] = df['BodyTemp'].astype(float)

In [ ]:
# Define conditions for HDP
def conditions(s):
    if (s['SystolicBP'] >= 140):
        return 'High probability of having HDP'
    else:
        return 'Less probability of having HDP'
    if (s['DiastolicBP'] >=90):
        return 'High probability of having HDP'
    else:
        return 'Less probability of having HDP'
    if (s['Age'] >= 35):
        return 'High probability of having HDP'
    else:
        return 'Less probability of having HDP'
    if (s['BS'] >= 6.7):
        return 'High probability of having HDP'
    else:
        return 'Less probability of having HDP'
    if (s['HeartRate'] >= 100):
        return 'High probability of having HDP'
    else:
        return 'Less probability of having HDP'
    if (s['BodyTemp'] >= 38.0):
        return 'High probability of having HDP'
    else:
        return 'Less probability of having HDP'

df['class'] = df.apply(conditions, axis=1)
df.head()

,Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate,RiskLevel,class
0,25.0,130.0,80.0,15.0,36.666667,86.0,2,Less probability of having HDP
1,35.0,140.0,90.0,13.0,36.666667,70.0,2,High probability of having HDP
2,29.0,90.0,70.0,8.0,37.777778,80.0,2,Less probability of having HDP
3,30.0,140.0,85.0,7.0,36.666667,70.0,2,High probability of having HDP
4,35.0,120.0,60.0,6.1,36.666667,76.0,0,Less probability of having HDP


In [ ]:
# Data summary
print(df.groupby('RiskLevel').size())
# Changed 'class' to 'RiskLevel'
print(df.isnull().sum())
print(df.dtypes)

RiskLevel
0    404
1    336
2    272
dtype: int64
Age            0
SystolicBP     0
DiastolicBP    0
BS             0
BodyTemp       0
HeartRate      0
RiskLevel      0
class          0
dtype: int64
Age            float64
SystolicBP     float64
DiastolicBP    float64
BS             float64
BodyTemp       float64
HeartRate      float64
RiskLevel        int64
class           object
dtype: object


In [ ]:
# Step 1: Split to X, Y
X = df[['SystolicBP', 'DiastolicBP', 'BS', 'BodyTemp', 'HeartRate', 'Age']].values
Y = df['RiskLevel'].values

In [ ]:
# Step 2: Train the model
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=42)

In [ ]:
# Cross-validation of models to find out which could perform better
models = [
    ('DTree', DecisionTreeClassifier()),
    ('Gaussian', GaussianNB()),
    ('KNN', KNeighborsClassifier()),
    ('LogisticReg', LogisticRegression(solver='liblinear', multi_class='ovr')),
    ('Gradient Boosting', GradientBoostingClassifier()),
    ('Linear Disc', LinearDiscriminantAnalysis())
]

for name, model in models:
    n_splits = min(10, len(X_train))  # Ensure n_splits does not exceed the number of samples
    kfold = KFold(n_splits=n_splits, random_state=42, shuffle=True)
    cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring='accuracy')
    print('Model Name ', name, 'Results', cv_results.mean())

Model Name  DTree Results 0.8234406438631791
Model Name  Gaussian Results 0.5974245472837021
Model Name  KNN Results 0.6794567404426559
Model Name  LogisticReg Results 0.6016700201207243
Model Name  Gradient Boosting Results 0.7768209255533199
Model Name  Linear Disc Results 0.6242857142857143


In [ ]:
# Pick GradientBoostingClassifier as it scored higher from the cross-validation
model = GradientBoostingClassifier()
model.fit(X_train, Y_train)

GradientBoostingClassifier()

In [ ]:
# Save the model for later use with Streamlit
joblib.dump(model, 'preeclampsia_model.pkl')

['preeclampsia_model.pkl']

In [ ]:
# Ask the model to predict 30% test data
predictions = model.predict(X_test)

In [ ]:
# Check accuracy
accuracy = accuracy_score(Y_test, predictions)
print('Accuracy: ', accuracy)

print('Report: ', classification_report(Y_test, predictions))
print(confusion_matrix(Y_test, predictions))

Accuracy:  0.7664473684210527
Report:                precision    recall  f1-score   support

           0       0.79      0.75      0.77       127
           1       0.68      0.70      0.69       105
           2       0.86      0.90      0.88        72

    accuracy                           0.77       304
   macro avg       0.77      0.78      0.78       304
weighted avg       0.77      0.77      0.77       304

[[95 29  3]
 [24 73  8]
 [ 1  6 65]]


In [ ]:
%%writefile app.py

Overwriting app.py


In [ ]:
! wget -q -O - ipv4.icanhazip.com

34.106.254.165


In [ ]:
!npm install -g localtunnel


changed 22 packages, and audited 23 packages in 2s

3 packages are looking for funding
  run `npm fund` for details

1 moderate severity vulnerability

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.


In [ ]:
! streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.106.254.165:8501

your url is: https://fifty-actors-hear.loca.lt
2024-08-27 14:02:56.739 Uncaught app exception
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/streamlit/runtime/scriptrunner/exec_code.py", line 85, in exec_func_with_error_handling
    result = func()
  File "/usr/local/lib/python3.10/dist-packages/streamlit/runtime/scriptrunner/script_runner.py", line 576, in code_to_exec
    exec(code, module.__dict__)
  File "/content/app.py", line 28, in <module>
    result = predict_risk_level(Age, SystolicBP, DiastolicBP, BS, BodyTemp, HeartRate)
  File "/content/app.py", line 10, in predict_risk_level
    if systolic_bp >= 140 or diastolic_bp >= 90:
NameError: name 'systolic_bp' is not defined
